In [1]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)

import fnmatch

import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

import seaborn as sns
import scipy as sp

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from scipy import stats

from aeon.transformations.collection.dictionary_based import PAA
import scipy.stats as stats

import sys
sys.path.append("../")

import warnings
warnings.filterwarnings("ignore")

/Users/bzcschae/miniconda3/lib/python3.10/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
path_queries = "queries/"

files_queries = {    
    "LenDB_queries.bin" : [256, np.float32],
    "SCEDC_queries.bin" : [256, np.float32],
    "Meier2019JGR_queries.bin" : [256, np.float32],
    "SIFT1b_queries.bin" : [256, np.float32],
    "OBS_queries.bin" : [256, np.float32],
    "bigANN_queries.bin" : [100, np.uint8],    
    "Iquique_queries.bin" : [256, np.float32],
    "astro_queries.bin": [256, np.float32],    
    "ETHZ_queries.bin" : [256, np.float32],
    "OBST2024_queries.bin" : [256, np.float32],
    "ISC_EHB_DepthPhases_queries.bin" : [256, np.float32],
    "NEIC_queries.bin" : [256, np.float32],
    "SALD_queries.bin" : [256, np.float32],    
    "deep1b_queries.bin" : [96, np.float32],    
    "PNW_queries.bin" : [256, np.float32],


    
    #"SimSearchNet_queries.bin" : [256, np.uint8],
    # "spacev1B_queries.bin" : [100, np.int8],
    #"TEXT_TO_IMAGE_queries.bin" : [200, np.float32],    
     # "turing_ANNS__head2.bin": [100, np.int8],    
    #"turingANNs_queries.bin": [100, np.float32],    
    #"SCEDC_queries.bin" : [256, np.float32],
    #"sift1b_queries.bin" : [128, np.float32],
    #"seismic_queries.bin" : [256, np.float32],
    #"noiseSeismic01.bin" : [256, np.float32],
    }

# "noiseSALD01.bin" : [128, np.float32],
# "noiseSeismic01.bin": [256, np.float32],    


path = "../data_head/"

files = {    
    "GEOFON_queries.bin" : [256, np.float32],
    "ETHZ_queries.bin" : [256, np.float32],
    #"SimSearchNet_head.bin" : [256, np.uint8],
    #"spacev1B_head.bin" : [100, np.int8],
    #"bigANN_head.bin" : [100, np.uint8],
    #"text-to-image_head.bin" : [200, np.float32],    
    #"turingANNs_head.bin": [100, np.int8],    
    #"astro_head.bin": [256, np.float32],
    #"SALD_head.bin" : [256, np.float32],
    #"SCEDC_head.bin" : [256, np.float32],
    #"sift1b_head.bin" : [128, np.float32],
    #"deep1b_head.bin" : [96, np.float32],
    #"seismic_head.bin" : [256, np.float32],
    }


In [3]:
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)

for i, key in enumerate(files_queries):    
    file = path_queries + key
    length, data_type = files_queries[key]

    data = np.fromfile(file, dtype=data_type)
    ## data = np.fromfile(file, dtype=np.float32)
    # data = (data[:len(data)-len(data)%length].reshape(-1, length).T)
    data = sp.stats.zscore(data)
    
    dataset = key.replace(".bin","") 
    variance = np.var(np.diff(data))
    print(f"Dataset: { dataset } \t { variance}")

Dataset: LenDB_queries 	 0.04207402467727661
Dataset: SCEDC_queries 	 0.48584306240081787
Dataset: Meier2019JGR_queries 	 0.0765201598405838
Dataset: SIFT1b_queries 	 1.4739598035812378
Dataset: OBS_queries 	 0.09686841070652008
Dataset: bigANN_queries 	 1.481627972408732
Dataset: Iquique_queries 	 0.6390228867530823
Dataset: astro_queries 	 1.8231006860733032
Dataset: ETHZ_queries 	 nan
Dataset: OBST2024_queries 	 0.47186943888664246
Dataset: ISC_EHB_DepthPhases_queries 	 0.023546593263745308
Dataset: NEIC_queries 	 0.15521876513957977
Dataset: SALD_queries 	 1.2995330095291138
Dataset: deep1b_queries 	 2.266934633255005
Dataset: PNW_queries 	 0.21480906009674072
